In [1]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer , BertTokenizer, GPT2PreTrainedModel, GPT2Model, GPT2Config, GPT2LMHeadModel
import torch
from typing import Optional, Tuple
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from typing import*
from torch import tensor

PRETRAINED_MODEL_NAME = "gpt2"
Padding_id = 50256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [27]:
df_train = pd.read_csv("/kaggle/input/asdfffff/SQuAD.csv", index_col=0)

In [28]:
len(df_train)

100000

In [29]:
df_train = df_train[df_train['text'].isna() == False]

In [30]:
df_train.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [31]:
df_train['text'] = df_train['text'].apply(lambda x : str(' ') + str(x) )
df_train.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [32]:
df_train = df_train.iloc[0:1000]

In [33]:
def my_pad_sequence(
    sequences: Union[tensor, List[tensor]],
    batch_first: bool = False,
    padding_value: int = 0,
    pos: str = 'right',
) -> tensor:

    if pos == 'right':
        padded_sequence = torch._C._nn.pad_sequence(sequences, batch_first, padding_value)
    elif pos == 'left':
        sequences = tuple(map(lambda s: s.flip(0), sequences))
        padded_sequence = torch._C._nn.pad_sequence(sequences, batch_first, padding_value)
        _seq_dim = padded_sequence.dim()
        padded_sequence = padded_sequence.flip(-_seq_dim+batch_first)
    else:
        raise ValueError("pos should be either 'right' or 'left', but got {}".format(pos))
    return padded_sequence


def QA_collate_batch(sample): #sample is List
    input_ids_batch = [s[0] for s in sample]
    label_batch = [s[1] for s in sample]

    input_ids_batch = my_pad_sequence(input_ids_batch, batch_first=True, padding_value = Padding_id, pos='left')
    label_batch = my_pad_sequence(label_batch, batch_first=True, padding_value = -100, pos='right')
    attn_batch = (input_ids_batch != Padding_id).int()

    return input_ids_batch, attn_batch, label_batch


class QAdataset(Dataset):
    def __init__(self, df, tokenizer):
        self.tokenizer = tokenizer
        self.df = df
    
    def __getitem__(self, index):
        df = self.df
        EC = self.tokenizer.encode_plus('context :' + df['context'][index] + 'question :' + df['question'][index] + 'answer :')
        input_ids = torch.tensor(EC['input_ids'], dtype=torch.long)
        
        ## ECl = self.tokenizer.encode_plus(df['label'][index])
        ECl = self.tokenizer.encode_plus(df['text'][index])
        label = torch.tensor(ECl['input_ids'], dtype=torch.long)

        return input_ids, label
    
    def __len__(self):
        return len(self.df)

In [34]:
tokenizer = GPT2Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer.pad_token_id = Padding_id

In [35]:
import pandas as pd 
from IPython.display import clear_output
from torch.utils.data import DataLoader
import random

Batch_size = 4
File_path = "/kaggle/input/asdfffff/SQuAD.csv"

df = df_train
ds = QAdataset(df, tokenizer) 
dataloader = DataLoader(ds, batch_size=Batch_size, shuffle=True, collate_fn=QA_collate_batch)

In [36]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        model = GPT2LMHeadModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.transformer = model.transformer
        self.lm_head = model.lm_head
        self.config = self.transformer.config
        # self.lm_head = torch.nn.Linear(self.config.n_embd, self.config.vocab_size, bias=False)
    
    def forward(self,
                input_ids,
                attention_mask,
                past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
                encoder_hidden_states: Optional[torch.Tensor] = None,
                encoder_attention_mask: Optional[torch.FloatTensor] = None,
                labels: Optional[torch.LongTensor] = None,
                ) :

        if labels is not None :
            predict_logits = self.trainmode(input_ids=input_ids,
                                            attention_mask=attention_mask,
                                            labels=labels
                                            )
            predict_words = predict_logits.argmax(dim=-1)  ## batch, predict_length
            
#             print(predict_words.size())'
#             print(labels.size())
            
#             print(tokenizer.convert_ids_to_tokens(predict_words[0]))
#             print(tokenizer.convert_ids_to_tokens(labels[0]))
            
            lossfn = torch.nn.CrossEntropyLoss()
            loss = lossfn(predict_logits.view(-1, predict_logits.size(-1)), labels.view(-1))

        return loss
    
    
    def trainmode(self,
                  input_ids,
                  attention_mask,
                  labels: Optional[torch.LongTensor] = None, ## batch, seqlen
                  ):
        padding_token_id = self.config.pad_token_id
        
        predict_logits = []
        
        batch_size = input_ids.size()[0]
        should_pred_len = labels.size()[1]
        curlen = 0

        unfinished_sentence = torch.tensor([1] * batch_size) ## batch_size
        ### when first run the sentence, there is no past_key_values
        ### so need the past_key_values for the setence, and then predict later
        
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        transformer_outputs = self.transformer(
            input_ids = input_ids,
            attention_mask = attention_mask
        )
        input_ids = input_ids.cpu()
        attention_mask = attention_mask.cpu()

        #### the three need return is (1)last_hidden (2)logits (3)past_key_values

        last_hidden = transformer_outputs.last_hidden_state ## batch size, seq_length, tensor_size
        logits = self.lm_head(last_hidden).cpu()  ## batch size, seq_length, vacabulary size
        last_hidden = last_hidden.cpu()
        past_key_values = transformer_outputs.past_key_values  ## layer, 2, batchsize, num_head, seq_len, one_head_len

        
        ##check print(labels)
        
        #### predict the label
        this_peer_finished = False
        while this_peer_finished == False:

            #### chose the predict logits from pre round #######################################################
            ####################################################################################################
            #### the logits_from_pre_step must not "all is eos"
            logits_from_pre_step = logits[:,-1,:]  ## batch, vacabulary size
            prob = torch.nn.functional.softmax(logits_from_pre_step, dim=-1)  ## batch, vocab_size
            selected_index = torch.argmax(prob, dim=-1)  ## batch
            # pred_tokens_id = prob[:, selected_index].unsqueeze(dim=-1)  ## batch , 1
            pred_tokens_id = selected_index.unsqueeze(dim=-1)  ## batch, 1 ~~ this is the new token id for each sequence
            
            predict_logits.append(logits_from_pre_step)
            ##############################################################################################
            
#           print(pred_tokens_id)
#             print(labels[:,curlen])

            #### prepare the new input, and the output for next round(step) ####################################
            ####################################################################################################
            #### random pick the next input
            rnd = torch.tensor([random.random() for _ in range(batch_size)])
            pick_answer = (rnd > 0.5).int()
            pick_answer = pick_answer * unfinished_sentence
            ### if has finished the sentence, then the label[k,cur_len] should be the -100, so cannot choose it
            pick_answer = pick_answer.unsqueeze(dim=-1)
            input_ids = (1-pick_answer) * pred_tokens_id + pick_answer * labels[:,curlen].unsqueeze(dim=-1)
            
#             print(pred_tokens_id)
#             print(input_ids)
            
            
            ##check print(pick_answer)
            ##check print(input_ids)
            
            # input_ids = pred_tokens_id
            #### 

            #### get new attention mask
            
            ##### all set to attention
            #### new_attention_mask = torch.tensor([[1]] * batch_size, dtype=torch.int32)
            
            #### after one eof appear in prediction, only attented the first eof, other is mask
            new_attention_mask = unfinished_sentence[:,None]
            # print(new_attention_mask)
            
            attention_mask = torch.cat([attention_mask, new_attention_mask], dim=-1) 
            ####
            
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            
            outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, past_key_values=past_key_values)
            
            input_ids = input_ids.cpu()
            attention_mask = attention_mask.cpu()
            ####################################################################################################

            #### renew the past key value, logits ##############################################################
            ####################################################################################################
            next_last_hidden = outputs.last_hidden_state  ## (batch, 1, hidden_size)
            logits = self.lm_head(next_last_hidden).cpu() ## (batch, 1, vocab_size)
            last_hidden = torch.cat([last_hidden, next_last_hidden.cpu()], dim=1)
            past_key_values = outputs.past_key_values
            #### 
            ####################################################################################################
    
            curlen += 1
            if curlen == should_pred_len:
                this_peer_finished = True
            else :
                unfinished_sentence = unfinished_sentence * labels[:,curlen].ne(torch.tensor([-100]) )

        ##############################################
        predict_logits = torch.stack(predict_logits)
        predict_logits = predict_logits.permute(1, 0, 2).contiguous() ## seq_len, batch, vocab_size -> batch, seq_len, vocab_size
        return predict_logits
            

In [ ]:
model = MyModel()
# model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(DEVICE)
model.config.pad_token_id = Padding_id

optimizer = torch.optim.SGD(model.parameters(), lr=0.00002, momentum=0.5, nesterov = True)

for _ in range(5) :
    myloss = 0

    for data in dataloader:
        input_id, attention_mask , label = data

    #     input_id = input_id.to(DEVICE)
    #     attention_mask = attention_mask.to(DEVICE)
    #     label = label.to(DEVICE)

        loss = model(input_ids = input_id, attention_mask = attention_mask, labels = label)
#         print(loss)
        
        myloss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(myloss / len(ds))


1378.26884496212
884.2304399013519
763.5755435675383
